In [1]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

from DePass import decomposed_state_manager
from DePass.utils import from_states_to_probs
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer,  Qwen2_5OmniForConditionalGeneration, Qwen2_5OmniProcessor
import plotly.io as pio
from plotly.subplots import make_subplots

model_path = "/ssd/linyiwu/Qwen2.5-Omni-7B/" # The path to model
model = Qwen2_5OmniForConditionalGeneration.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation='flash_attention_2'
    
)
# tokenizer = AutoTokenizer.from_pretrained(model_name)
model = model.eval() 
processor = Qwen2_5OmniProcessor.from_pretrained(model_path)

/home/linyiwu/anaconda3/envs/omnizip/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
Qwen2_5OmniToken2WavModel must inference with fp32, but flash_attention_2 only supports fp16 and bf16, attention implementation of Qwen2_5OmniToken2WavModel will fallback to sdpa.
Loading checkpoint shards: 100%|██████████| 5/5 [00:04<00:00,  1.01it/s]
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading fro

In [2]:
sys.path.append("/home/linyiwu/OmniZip")
from omnizip.note_modeling_qwen2_5_omni import (
    Qwen2_5OmniAttention,       # Eager
    Qwen2_5OmniFlashAttention2, # Flash Attention 2
    Qwen2_5OmniSdpaAttention    # SDPA
)
def swap_attention_module(module, layer_idx, replacing_attn):
    old_attn = module.self_attn
    old_config = module.self_attn.config
    current_device = old_attn.q_proj.weight.device
    current_dtype = old_attn.q_proj.weight.dtype

    new_attn = replacing_attn(old_config, layer_idx=layer_idx)
    new_attn.load_state_dict(old_attn.state_dict())
    new_attn.to(current_device, dtype=current_dtype)
    module.self_attn = new_attn
    print(f"Layer {layer_idx} attention swapped to {replacing_attn.__name__}")


for layer_idx, layer in enumerate(model.thinker.model.layers):
    swap_attention_module(layer, layer_idx, Qwen2_5OmniSdpaAttention)

Layer 0 attention swapped to Qwen2_5OmniSdpaAttention
Layer 1 attention swapped to Qwen2_5OmniSdpaAttention
Layer 2 attention swapped to Qwen2_5OmniSdpaAttention
Layer 3 attention swapped to Qwen2_5OmniSdpaAttention
Layer 4 attention swapped to Qwen2_5OmniSdpaAttention
Layer 5 attention swapped to Qwen2_5OmniSdpaAttention
Layer 6 attention swapped to Qwen2_5OmniSdpaAttention
Layer 7 attention swapped to Qwen2_5OmniSdpaAttention
Layer 8 attention swapped to Qwen2_5OmniSdpaAttention
Layer 9 attention swapped to Qwen2_5OmniSdpaAttention
Layer 10 attention swapped to Qwen2_5OmniSdpaAttention
Layer 11 attention swapped to Qwen2_5OmniSdpaAttention
Layer 12 attention swapped to Qwen2_5OmniSdpaAttention
Layer 13 attention swapped to Qwen2_5OmniSdpaAttention
Layer 14 attention swapped to Qwen2_5OmniSdpaAttention
Layer 15 attention swapped to Qwen2_5OmniSdpaAttention
Layer 16 attention swapped to Qwen2_5OmniSdpaAttention
Layer 17 attention swapped to Qwen2_5OmniSdpaAttention
Layer 18 attention s

In [3]:
# prepare data

video_path = "/ssd/linyiwu/worldsense/videos/FnOIAada.mp4"
question = "What is the number on the rear wing of the red vehicle at the start of the video?"
task_candidates = [               
    "A. 56.",
    "B. 50.",
    "C. 58.",
    "D. 59."
]
conversation = [
    {"role": "system", "content": [{"type": "text", "text": "You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech."}]},
    {
        "role": "user",
        "content": [
            {"type": "video", "video": video_path},
            {"type": "text", "text": f"{question}\nChoose from the following options:\n" + "\n".join(task_candidates) + "\nPlease provide your answer by selecting the corresponding option and give your answer straight away."},
        ],
    },
]


In [4]:
from DePass import qwenOmnimanager
# print(type(decomposed_state_manager))
# print(type(qwenOmnimanager))
# with open("/home/linyiwu/OmniZip/print_result.md", "w") as f:
#     print(model.thinker.config.text_config, file=f)

DecomposedStateManager = qwenOmnimanager(model, processor, mlp_decomposed_function="softmax")

In [5]:
model.thinker.model.config

Qwen2_5OmniTextConfig {
  "attention_dropout": 0.0,
  "hidden_act": "silu",
  "hidden_size": 3584,
  "init_std": 0.02,
  "initializer_range": 0.02,
  "intermediate_size": 18944,
  "max_position_embeddings": 32768,
  "max_window_layers": 28,
  "model_type": "qwen2_5_omni_text",
  "num_attention_heads": 28,
  "num_hidden_layers": 28,
  "num_key_value_heads": 4,
  "rms_norm_eps": 1e-06,
  "rope_scaling": {
    "mrope_section": [
      16,
      24,
      24
    ],
    "rope_type": "default",
    "type": "default"
  },
  "rope_theta": 1000000.0,
  "sliding_window": 32768,
  "tie_word_embeddings": false,
  "transformers_version": "4.52.3",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 152064
}

In [6]:
states, attribute_state, mod_names = DecomposedStateManager.get_last_layer_decomposed_state(conversation)


['<|im_start|>system\nYou are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech.<|im_end|>\n<|im_start|>user\n<|vision_bos|><|VIDEO|><|vision_eos|>What is the number on the rear wing of the red vehicle at the start of the video?\nChoose from the following options:\nA. 56.\nB. 50.\nC. 58.\nD. 59.\nPlease provide your answer by selecting the corresponding option and give your answer straight away.<|im_end|>\n<|im_start|>assistant\n']


/home/linyiwu/OmniZip/qwen-omni-utils/src/qwen_omni_utils/v2_5/audio_process.py:85: UserWarning: PySoundFile failed. Trying audioread instead.
  librosa.load(
/home/linyiwu/anaconda3/envs/omnizip/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
qwen-vl-utils using decord to read video.
Unused or unrecognized kwargs: return_tensors, images.
Computing Audio:   0%|          | 0/600 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.56 GiB. GPU 0 has a total capacity of 23.52 GiB of which 1.42 GiB is free. Including non-PyTorch memory, this process has 22.07 GiB memory in use. Of the allocated memory 21.02 GiB is allocated by PyTorch, and 610.74 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
last_layer_state = states[-1]
state = DecomposedStateManager.text_model.norm(last_layer_state[0, -1, :])
def from_states_to_probs(manager, states, topk=5):
    """convert states(d dimension)to token ids and probabilities(vocabulary size)
    Args:
        states: torch.Tensor, shape=(d, )
        tokenizer: transformers.Tokenizer
        layer_idx: int, layer index
        topk: int, topk
    Returns:
        token_probs: dict, token probabilities
    """
    logits = manager.model.thinker.lm_head(states)
    traj_log_probs = logits.log_softmax(dim=-1).squeeze()
    topk_values, topk_indices = torch.topk(traj_log_probs, k=topk)
    probs = torch.exp(topk_values)
    token_probs = []
    for idx, prob in zip(topk_indices.cpu(), probs.cpu()):
        token = manager.processor.batch_decode(
                    [idx],
                )

        token_probs.append((idx.item(), token, prob.item()))
    return token_probs

probs = from_states_to_probs(DecomposedStateManager, state, topk=10)
probs


[(35, ['D'], 0.1572265625),
 (34, ['C'], 0.138671875),
 (11908, ['Oh'], 0.12890625),
 (80022, ['Hmm'], 0.12890625),
 (32, ['A'], 0.12890625),
 (785, ['The'], 0.08349609375),
 (2132, ['It'], 0.07373046875),
 (33, ['B'], 0.0693359375),
 (24765, ['Ah'], 0.0306396484375),
 (18665, ['Hey'], 0.02392578125)]

: 

: 

: 

: 